In [25]:
import os, sys
import numpy as np
import h5py, yaml
from pybird.io_pb import ReadWrite

In [26]:
import importlib, pybird
importlib.reload(pybird.io_pb)
from pybird.io_pb import ReadWrite

Loading data in .npy format

In [27]:
rootdir = os.path.join(os.path.expanduser('~'), 'Documents', 'github', 'pybird')
configdir = os.path.join(rootdir, 'data', 'eftboss', 'config')

In [28]:
config = yaml.full_load(open(os.path.join(configdir, 'boss_pk_wc.yaml'), 'r'))
config["data_path"] = os.path.join(rootdir, config["data_path"])
config["write"]["out_path"] = os.path.join(rootdir, config["write"]["out_path"])
print ( yaml.dump(config, indent=4, sort_keys=False))
config["data_file"] = "boss_dr12_2pt.yaml"

data_path: /Users/pierre/Documents/github/pybird/data/eftboss/
data_file: boss_dr12_2pt.h5
output: bPk
multipole: 2
with_wedge: false
wedge_type: PA-w1-w2
with_ap: true
with_survey_mask: true
with_bao_rec: true
sky:
    cmass_ngc:
        min:
        - 0.01
        - 0.01
        max:
        - 0.23
        - 0.23
    cmass_sgc:
        min:
        - 0.01
        - 0.01
        max:
        - 0.23
        - 0.23
    lowz_ngc:
        min:
        - 0.01
        - 0.01
        max:
        - 0.2
        - 0.2
    lowz_sgc:
        min:
        - 0.01
        - 0.01
        max:
        - 0.2
        - 0.2
write:
    save: true
    fake: false
    plot: true
    show: true
    out_path: /Users/pierre/Documents/github/pybird/data/eftboss/out/
    out_name: boss_pk_wc
km: 0.7
kr: 0.35
nd: 0.0004
eft_basis: westcoast
with_stoch: true
with_nnlo_counterterm: false
eft_prior:
    b1:
        type: flat
        range: null
    c2:
        type: gauss
        mean:
        - 0.0
        - 0.0


In [29]:
io = ReadWrite()

In [30]:
d, _, _ = io.read(config, verbose=True) 

reading data file: /Users/pierre/Documents/github/pybird/data/eftboss/boss_dr12_2pt.h5
-----------------------
sky: cmass_ngc
output: bPk
multipole: 2
min bound (per multipole): [0.01, 0.01]
max bound (per multipole): [0.23, 0.23]
bao rec: on
coordinate (AP) distortion: on
survey mask: on
-----------------------
-----------------------
sky: cmass_sgc
output: bPk
multipole: 2
min bound (per multipole): [0.01, 0.01]
max bound (per multipole): [0.23, 0.23]
bao rec: on
coordinate (AP) distortion: on
survey mask: on
-----------------------
-----------------------
sky: lowz_ngc
output: bPk
multipole: 2
min bound (per multipole): [0.01, 0.01]
max bound (per multipole): [0.2, 0.2]
bao rec: on
coordinate (AP) distortion: on
survey mask: on
-----------------------
-----------------------
sky: lowz_sgc
output: bPk
multipole: 2
min bound (per multipole): [0.01, 0.01]
max bound (per multipole): [0.2, 0.2]
bao rec: on
coordinate (AP) distortion: on
survey mask: on
-----------------------
cmass_ngc: 

```d``` is a python nested dictionary

Saving it to hdf5

In [31]:
def save_dict_to_hdf5(group, data, none_flag="NoneValue"):
    def save_recursive(subgroup, subdata):
        for key, value in subdata.items():
            if isinstance(value, dict):
                save_recursive(subgroup.create_group(key), value)
            elif value is None:
                subgroup.create_dataset(key, data=none_flag)
            else:
                subgroup.create_dataset(key, data=value)
    
    save_recursive(group, data)

In [32]:
h5filename = os.path.join(config["data_path"], '%s.h5') % config["data_file"].split('.', 1)[0]
print (h5filename)

/Users/pierre/Documents/github/pybird/data/eftboss/boss_dr12_2pt.h5


In [24]:
with h5py.File(h5filename, 'w') as hf: save_dict_to_hdf5(hf, d)

Get back the dictionary from hdf5

In [107]:
def get_dict_from_hdf5(group, none_flag="NoneValue"):
    d = {}
    for key in group.keys():
        if isinstance(group[key], h5py.Group):
            d[key] = get_dict_from_hdf5(group[key], none_flag=none_flag)
        elif isinstance(group[key], h5py.Dataset):
            data = group[key][()]
            if np.array_equal(data, none_flag):
                d[key] = None
            else:
                d[key] = data
    return d

In [108]:
with h5py.File(h5filename, 'r') as hf: d2 = get_dict_from_hdf5(hf)